<center><big><b>STAT 906 A3</b></big></center>

##### Question 1

##### (a)

In [18]:
import numpy as np
from scipy.stats.mstats import gmean
from scipy.stats import norm

In [29]:
def asian_theo(r, T, d, S, K, sigma):
    a = np.log(S)+(r-0.5*sigma**2)*(d+1)*T/(2*d)
    b = sigma**2*(T/d)*(d+1)*(2*d+1)/(6*d)
    d1 = (-np.log(K)+a+b)/np.sqrt(b)
    d2 = d1-np.sqrt(b)
    return np.exp(-r*T)*(np.exp(a+0.5*b)*norm.cdf(d1)-K*norm.cdf(d2))

def beta_pilot(r, T, sigma, S, K, s, n):
    delta = T/s
    S_t = np.zeros((n, s+1))
    S_t[:,0] = S  
    for i in range(s):       # simulate the stock price
        S_t[:,(i+1)] = S_t[:,i]*np.exp((r-sigma**2/2)*delta+sigma*np.sqrt(delta)*np.random.normal(0,1,n))
    payoff_ath = np.mean(S_t[:,1:], axis=1) - K   
    payoff_ath = np.exp(-r*T)*np.where(payoff_ath > 0, payoff_ath, 0)     
    payoff_geo = gmean(S_t[:,1:], axis=1) - K
    payoff_geo = np.exp(-r*T)*np.where(payoff_geo > 0, payoff_geo, 0)
    cov_mat = np.cov(np.vstack((payoff_ath, payoff_geo)), bias=False)
    return cov_mat[1,0]/cov_mat[1,1]
    
def asian_cv_pilot(r, T, sigma, S, K, s, n, n_pilot):
    delta = T/s
    S_t = np.zeros((n, s+1))
    S_t[:,0] = S  
    for i in range(s):       # simulate the stock price
        S_t[:,(i+1)] = S_t[:,i]*np.exp((r-sigma**2/2)*delta+sigma*np.sqrt(delta)*np.random.normal(0,1,n))
    payoff_ath = np.mean(S_t[:,1:], axis=1) - K   
    payoff_ath = np.exp(-r*T)*np.where(payoff_ath > 0, payoff_ath, 0)     
    payoff_geo = gmean(S_t[:,1:], axis=1) - K
    payoff_geo = np.exp(-r*T)*np.where(payoff_geo > 0, payoff_geo, 0)
    beta = beta_pilot(r, T, sigma, S, K, s, n_pilot)
    C = asian_theo(r, T, s, S, K, sigma)
    mu_cv = payoff_ath + beta*(C - payoff_geo)
    asian_mean = np.mean(mu_cv)
    asian_var = np.var(mu_cv, ddof=1)
    hw = norm.ppf(0.975)*np.sqrt(asian_var/n)
    return [asian_mean, asian_mean - hw, asian_mean + hw, hw]

In [30]:
asian_cv_pilot(0.04, 1, 0.35, 50, 45, 10, 24000, 1000)

[7.572543648291707, 7.567629800415124, 7.57745749616829, 0.0049138478765831]

##### (b)

In [24]:
def asian_cv(r, T, sigma, S, K, s, n, m):
    delta = T/s
    S_t = np.zeros((n, s+1))
    S_t[:,0] = S  
    mu_cv = np.zeros(m)
    C = asian_theo(r, T, s, S, K, sigma)
    for j in range(m):
        for i in range(s):       # simulate the stock price
            S_t[:,(i+1)] = S_t[:,i]*np.exp((r-sigma**2/2)*delta+sigma*np.sqrt(delta)*np.random.normal(0,1,n))
        payoff_ath = np.mean(S_t[:,1:], axis=1) - K   
        payoff_ath = np.exp(-r*T)*np.where(payoff_ath > 0, payoff_ath, 0)     
        payoff_geo = gmean(S_t[:,1:], axis=1) - K
        payoff_geo = np.exp(-r*T)*np.where(payoff_geo > 0, payoff_geo, 0)
        cov_mat = np.cov(np.vstack((payoff_ath, payoff_geo)), bias=False)
        beta = cov_mat[1,0]/cov_mat[1,1]
        mu_cv[j] = np.mean(payoff_ath+beta*(C-payoff_geo))
    asian_mean = np.mean(mu_cv)
    asian_var = np.var(mu_cv, ddof=1)
    hw = norm.ppf(0.975)*np.sqrt(asian_var/m)
    return [asian_mean, asian_mean - hw, asian_mean + hw]

In [25]:
asian_cv(0.04, 1, 0.35, 50, 45, 10, 2400, 1000)

[7.5741457414498035, 7.573661684791208, 7.574629798108399]

##### (c)

In [31]:
def asian_mc(r, T, sigma, S, K, s, n):
    delta = T/s
    S_t = np.zeros((n, s+1))
    S_t[:,0] = S
    for i in range(s):
        S_t[:,(i+1)] = S_t[:,i]*np.exp((r-sigma**2/2)*delta+sigma*np.sqrt(delta)*np.random.normal(0,1,n))
    payoff = np.mean(S_t[:,1:], axis=1) - K 
    payoff = np.exp(-r*T)*np.where(payoff > 0, payoff, 0)
    asian_var = np.var(payoff, ddof=1)
    hw = norm.ppf(0.975)*np.sqrt(asian_var/n)
    return hw

In [34]:
_, _, _, hw45_cv = asian_cv_pilot(0.04, 1, 0.35, 50, 45, 10, 24000, 1000)
_, _, _, hw50_cv = asian_cv_pilot(0.04, 1, 0.35, 50, 50, 10, 24000, 1000)
_, _, _, hw60_cv = asian_cv_pilot(0.04, 1, 0.35, 50, 60, 10, 24000, 1000)
hw_45 = asian_mc(0.04, 1, 0.35, 50, 45, 10, 24000)
hw_50 = asian_mc(0.04, 1, 0.35, 50, 50, 10, 24000)
hw_60 = asian_mc(0.04, 1, 0.35, 50, 60, 10, 24000)
factor_45 = 1 - hw45_cv/hw_45
factor_50 = 1 - hw50_cv/hw_50
factor_60 = 1 - hw60_cv/hw_60
print(factor_45, factor_50, factor_60)

0.9576452879525877 0.9510899140224142 0.9267625359805669


##### Question 2

##### (a)

First of all , I need to simulate the stock price $S_i(t)$. To get that, I firstly need to simulte $N_i(t)$ and $W_i(t)$, $i = 1, 2, ..., n$, where $N_i(t)$s i.i.d. $\sim Poisson(0.5)$ and $W_i(t)$s i.i.d. $\sim Normal(0, 0.5)$. For each $i$, I simulate $Y_{ij}$ of number $N_i(t)$, where $Y_{ij}$s are i.i.d. $\sim Gamma(2, 0.5)$, and $j = 1, 2, ..., N_i(t)$. Then I could generate the $S_i(t)$ by $S_i(t) = S(0)e^{(\mu-\sigma^2/2)t+\sigma W_i(t)} \prod_{j=1}^{N_i(t)}Y_{ij}(t)$    

Let $X_{1} \sim Bernoulli(0.75)$ and $X_{2} \sim Bernoulli(0.2)$, and

$$\mu = E[X_{1} \mathbb{1}_{\{S(t)>120\}} + X_{2} \mathbb{1}_{\{S(t) \leq 120\}}] = E[E[X_{1} \mathbb{1}_{\{S(t)>120\}} + X_{2} \mathbb{1}_{\{S(t) \leq 120\}}]|S(t)=s]$$

Then I let $\hat{\mu}$ be the Monte Carlo estimator for the probability of selling the stock at $T = 1$, and 
$$\hat{\mu} = \frac{1}{n}\sum_{i=1}^n X_{i1} \mathbb{1}_{\{S_i(t)>120\}} + X_{i2} \mathbb{1}_{\{S_i(t) \leq 120\}}$$
where $X_{i1} \sim Bernoulli(0.75)$ and $X_{i2} \sim Bernoulli(0.2)$ for $i = 1, 2, ..., n$.

Finaly, I construct a conditional Monte Carlo estimator $\hat{\mu}_{cmc}$ for the probability of selling the stock at $T = 1$, and 
$$\hat{\mu}_{cmc} = \frac{1}{n}\sum_{i=1}^n E[X_{i1} \mathbb{1}_{\{S_i(t)>120\}} + X_{i2} \mathbb{1}_{\{S_i(t) \leq 120\}}|S_i(t)] = \frac{1}{n}\sum_{i=1}^n 0.75 \mathbb{1}_{\{S_i(t)>120\}} + 0.2 \mathbb{1}_{\{S_i(t) \leq 120\}}$$

##### (b)

In [62]:
def prob_mc(mu, sigma, a, b, Lambda, t, S, n):
    N_t = np.random.poisson(Lambda*t, n)
    Y = np.ones(n)
    for i in range(n):
        Y[i] = np.prod(np.random.gamma(a,b,N_t[i]))
    S_t = S*np.exp((mu-sigma**2/2)*t+np.random.normal(0,np.sqrt(t),n)*sigma)*Y  
    X1 = np.random.binomial(1, 0.75, n)
    X2 = np.random.binomial(1, 0.2, n)
    p_mc = np.where(S_t > 120, X1, X2)
    return np.mean(p_mc)

In [63]:
def prob_cmc(mu, sigma, a, b, Lambda, t, S, n):
    N_t = np.random.poisson(Lambda*t, n)
    Y = np.ones(n)
    for i in range(n):
        Y[i] = np.prod(np.random.gamma(a,b,N_t[i]))
    S_t = S*np.exp((mu-sigma**2/2)*t+np.random.normal(0,np.sqrt(t),n)*sigma)*Y  
    p_cmc = np.where(S_t > 120, 0.75, 0.2)
    return np.mean(p_cmc)

In [68]:
p_cmc = prob_cmc(0.07, 0.3, 2, 0.5, 1, 0.5, 100, 1000)
p_mc = prob_mc(0.07, 0.3, 2, 0.5, 1, 0.5, 100, 1000)
print(p_cmc, p_mc)

0.34300000000000014 0.379


##### Question 3

In [122]:
def put_str(mu, sigma, a, b, Lambda, t, S, K, n):
    p1 = np.exp(-Lambda*t)
    p2 = np.exp(-Lambda*t)*(Lambda*t)
    p3 = 1 - p1 - p2
    N_1 = int(n*p1)
    N_2 = int(n*p2)
    N_3 = np.max(n-N_1-N_2, 0)
    N_t = np.zeros(n, dtype=int)
    N_t[:N_1] = 0
    N_t[N_1:(N_1+N_2)] = 1
    Y = np.ones(n)
    for j in range(N_1+N_2, N_1+N_2+N_3):
        while True:
            temp = np.random.poisson(Lambda*t)
            if temp >= 2:
                N_t[j] = temp
                break;
    for i in range(n):
        Y[i] = np.prod(np.random.gamma(a,b,N_t[i]))
    S_t = S*np.exp((mu-sigma**2/2)*t+np.random.normal(0,np.sqrt(t),n)*sigma)*Y 
    payoff = np.exp(-mu*t)*np.where(K-S_t>0, K-S_t, 0)
    mu_1 = np.mean(payoff[:N_1])
    mu_2 = np.mean(payoff[N_1:(N_1+N_2)])
    mu_3 = np.mean(payoff[(N_1+N_2):(N_1+N_2+N_3)])
    mu = p1*mu_1 + p2*mu_2 + p3*mu_3
    hw = norm.ppf(0.975)*np.sqrt(np.var(payoff, ddof=1)/n)
    return [mu, hw]

In [123]:
put_str(0.07, 0.3, 2, 0.5, 1, 0.5, 100, 120, 1000)

[27.944130714225246, 1.7920225294349243]

In [124]:
def put_mc(mu, sigma, a, b, Lambda, t, S, K, n):
    N_t = np.random.poisson(Lambda*t, n)
    Y = np.ones(n)
    for i in range(n):
        Y[i] = np.prod(np.random.gamma(a,b,N_t[i]))
    S_t = S*np.exp((mu-sigma**2/2)*t+np.random.normal(0,np.sqrt(t),n)*sigma)*Y  
    payoff = np.exp(-mu*t)*np.where(K-S_t>0, K-S_t, 0)
    payoff_mean = np.mean(payoff)
    hw = norm.ppf(0.975)*np.sqrt(np.var(payoff, ddof=1)/n)
    return [payoff_mean, hw]

In [125]:
put_mc(0.07, 0.3, 2, 0.5, 1, 0.5, 100, 120, 1000)

[28.176616339911458, 1.7455988321159706]

##### Question 4

In [192]:
def asian_put_mc(r, T, sigma, s, S, K, n):
    delta = T/s
    S_t = np.zeros((n, s+1))
    S_t[:,0] = S
    for i in range(s):
        S_t[:,(i+1)] = S_t[:,i]*np.exp((r-sigma**2/2)*delta+sigma*np.sqrt(delta)*np.random.normal(0,1,n))
    payoff = K - np.mean(S_t[:,1:], axis=1)
    payoff = np.exp(-r*T)*np.where(payoff > 0, payoff, 0)
    C = np.mean(payoff)
    hw = norm.ppf(0.975)*np.sqrt(np.var(payoff, ddof=1)/n)
    return [C, C-hw, C+hw]

In [193]:
asian_put_mc(0.04, 1, 0.35, 10, 50, 45, 10000)

[1.7000328337197839, 1.6344865146419074, 1.7655791527976603]

In [161]:
def asian_put_is(r, T, sigma, s, S, K, n):
    delta = T/s
    S_t = np.zeros((n, s+1))
    L = np.zeros((n, s))
    S_t[:,0] = S
    for i in range(s):
        z = np.random.normal(0,1,n)
        S_t[:,(i+1)] = S_t[:,i]*np.exp((r-0.03-sigma**2/2)*delta+sigma*(z*np.sqrt(delta)))
        L[:,i] = np.exp(2*0.03*delta*z+(-0.03*delta)**2)                             
    payoff = K - np.mean(S_t[:,1:], axis=1)
    payoff = np.exp(-r*T)*np.where(payoff > 0, payoff, 0)*np.prod(L, axis=1)
    C = np.mean(payoff)
    hw = norm.ppf(0.975)*np.sqrt(np.var(payoff, ddof=1)/n)
    return [C, C-hw, C+hw]

In [162]:
asian_put_is(0.04, 1, 0.35, 10, 50, 45, 100)

[1.8208858241973227, 1.1662294564120144, 2.475542191982631]

##### (c)

In [ ]:
def asian_put_is(r, T, sigma, s, S, K, n):
    delta = T/s
    S_t = np.zeros((n, s+1))
    L = np.zeros((n, s))
    S_t[:,0] = S
    for i in range(s):
        z = np.random.normal(0,1,n)
        S_t[:,(i+1)] = S_t[:,i]*np.exp((r-sigma**2/2)*delta+sigma*(z*np.sqrt(delta)-0.03*delta))
        L[:,i] = np.exp(0.03*delta*z+(-0.03*delta)**2/2)                             
    payoff = K - np.mean(S_t[:,1:], axis=1)
    payoff = np.exp(-r*T)*np.where(payoff > 0, payoff, 0)*np.prod(L, axis=1)
    C = np.mean(payoff)
    hw = norm.ppf(0.975)*np.sqrt(np.var(payoff, ddof=1)/n)
    return [C, C-hw, C+hw]

##### Question 5

For naive Monte Carlo method, we have   

$
\begin{split}
Var(\hat{\mu}_{mc}) 
&= Var(\frac{1}{n}\sum_{i=1}^{n} h(X_i)) \\
&= \frac{1}{n^2} \sum_{i=1}^{n} Var(h(X_i)) \\
&= \frac{1}{n} Var(h(X))
\end{split}
$

For estimator using stratification with proportional allocation, we have 

$
\begin{split}
Var(\hat{\mu}_{str})
&= Var(\sum_{j=1}^m p_j \frac{1}{N_j} \sum_{i=1}^{N_j} h(X_{j,i})) \\
&= \sum_{j=1}^m Var(p_j \frac{1}{np_j} \sum_{i=1}^{np_j} h(X_{j,i})) \\
&= \sum_{j=1}^m \frac{1}{n^2} \sum_{i=1}^{np_j} Var(h(X_{j,i})) \\ 
&= \frac{1}{n} \sum_{j=1}^m p_j Var(E[h(X)|W\in S_j]) \\
\end{split}
$

Since $Var(h(X)) = Var(E[E[h(X)|W\in S_j]]) + E[Var(h(X)|W\in S_j)]$ holds for any $j = 1, 2, ..., m$, then we have $Var(E[h(X)|W\in S_j]) \leq Var(h(X))$ holds for any $j = 1, 2, ..., m$. Note that $Var(h(X)|W\in S_j) \geq 0$ for any $j = 1, 2, ..., m$. Therefore,

$
\begin{split}
Var(\hat{\mu}_{str})
& \leq \frac{1}{n} \sum_{j=1}^m p_j Var(h(X)) \\
&= \frac{1}{n} Var(h(X)) \sum_{j=1}^m p_j \\
&= \frac{1}{n} Var(h(X)) \\
&= Var(\hat{\mu}_{mc})
\end{split}
$

So we can conclude that estimator using stratification with proportional allocation has a variance no larger than that of naive Monte Carlo estimator. The equality holds when $Var(h(X)|W\in S_j) = 0$ for any $j = 1, 2, ..., m$

##### Question 6

##### (a)

In [170]:
def Delta(r, T, sigma, h, S, K, n):
    z = np.exp((r-sigma**2/2)*T+sigma*np.random.normal(0,np.sqrt(T),n))
    payoff = S*z - K
    price = np.exp(-r*T)*np.where(payoff > 0, payoff, 0)
    payoff_h = (S+h)*z - K
    price_h = np.exp(-r*T)*np.where(payoff_h > 0, payoff_h, 0)
    delta = (price_h - price)/h
    delta_mean = np.mean(delta)
    hw = norm.ppf(0.975)*np.sqrt(np.var(delta, ddof=1)/n)  
    return [delta_mean, delta_mean-hw, delta_mean+hw]   

In [171]:
Delta(0.04, 1, 0.25, 1, 100, 90, 10000)

[0.7622742236897467, 0.7514661693114317, 0.7730822780680617]

##### (b)

In [168]:
def Delta_ind(r, T, sigma, h, S, K, n):
    S_t = S*np.exp((r-sigma**2/2)*T+sigma*np.random.normal(0,np.sqrt(T),n))
    payoff = S_t - K
    price = np.exp(-r*T)*np.where(payoff > 0, payoff, 0)
    S_t_h = (S+h)*np.exp((r-sigma**2/2)*T+sigma*np.random.normal(0,np.sqrt(T),n))
    payoff_h = S_t_h - K
    price_h = np.exp(-r*T)*np.where(payoff_h > 0, payoff_h, 0)
    delta = (price_h - price)/h
    delta_mean = np.mean(delta)
    hw = norm.ppf(0.975)*np.sqrt(np.var(delta, ddof=1)/n)  
    return [delta_mean, delta_mean-hw, delta_mean+hw]  

In [174]:
Delta_ind(0.04, 1, 0.25, 1, 100, 90, 10000)

[0.6030370859647095, 0.015906596239594784, 1.1901675756898242]

##### (c)

In [175]:
from scipy.stats import norm

In [177]:
def Delta_theo(r, T, sigma, S, K):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T)/(sigma*np.sqrt(T))
    return norm.cdf(d1)

In [178]:
Delta_theo(0.04, 1, 0.25, 100, 90)

0.7600433646090563

##### (d)

In [179]:
Delta(0.04, 1, 0.25, 0.1, 100, 90, 10000)

[0.7609687748765028, 0.7501123038482689, 0.7718252459047368]

In [182]:
Delta(0.04, 1, 0.25, 0.01, 100, 90, 10000)

[0.7606308510655381, 0.7497311456833352, 0.7715305564477409]

In [188]:
Delta_ind(0.04, 1, 0.25, 0.1, 100, 90, 10000)

[-0.586859154885877, -6.454122304735633, 5.280403994963879]

In [191]:
Delta_ind(0.04, 1, 0.25, 0.01, 100, 90, 10000)

[23.0785703997818, -35.70117992991861, 81.8583207294822]